In [1]:
import os

In [2]:
%pwd 

'd:\\Python Hub\\MLOPS\\End-to-End-ML-Approach-to-Wine-Quality-Prediction-with-MLflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Python Hub\\MLOPS\\End-to-End-ML-Approach-to-Wine-Quality-Prediction-with-MLflow'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/pritamnarwade11/End-to-End-ML-Approach-to-Wine-Quality-Prediction-with-MLflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="pritamnarwade11"
os.environ["MLFLOW_TRACKING_PASSWORD"]="871b76c6a3213893886036cf5681b2454522ec97"  

In [6]:
from dataclasses import dataclass
from pathlib import Path 


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path 
    test_data_path: Path 
    model_path: Path 
    all_params: dict 
    metric_file_name: Path 
    target_column: str 
    mlflow_uri: str 

In [12]:
from MLproject.constants import *
from MLproject.utils.common import read_yaml, create_directories, save_json

In [14]:
class ConfigurationManager: 
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH ):
        
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation 
        schema = self.schema.TARGET_COLUMN
        params = self.params.ElasticNet 

        create_directories([config.root_dir])    

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            all_params= params,
            metric_file_name= config.metric_file_name,
            target_column= schema.name, 
            mlflow_uri= "https://dagshub.com/pritamnarwade11/End-to-End-ML-Approach-to-Wine-Quality-Prediction-with-MLflow.mlflow"
        )

        return model_evaluation_config

In [9]:
import os 
import pandas as pd 
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score 
from urllib.parse import urlparse 
import mlflow 
import mlflow.sklearn 
import joblib

In [10]:
class ModelEvaluation:
    def __init__(self, config : ModelEvaluationConfig):
        self.config = config 


    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred) 
        r2 = r2_score(actual, pred) 
        return rmse, mae, r2 
    
    
    def log_into_mlflow(self):

        test_data = np.load(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data[:, :-1]
        y_test = test_data[:, -1]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme 

        # mlflow
        with mlflow.start_run():

            predicted_qualities = model.predict(X_test)

            (rmse, mae, r2) = self.eval_metrics(y_test, predicted_qualities)

            # savings metrics as local 
            scores = {"rmse": rmse, "mae": mae, "r2":r2}
            save_json(path=Path(self.config.metric_file_name), data= scores)
            
            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(
                    model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
    

In [15]:
try:
    config = ConfigurationManager()
    model_evalutaion_config = config.get_model_evaluation_config()
    model_config = ModelEvaluation(config=model_evalutaion_config)
    model_config.log_into_mlflow()
except Exception as e:
    raise e     

[ 2023-12-30 12:35:19,850:  INFO:  common: 33:  YAML file 'config\config.yaml' loaded successfully]
[ 2023-12-30 12:35:19,859:  INFO:  common: 33:  YAML file 'schema.yaml' loaded successfully]
[ 2023-12-30 12:35:19,865:  INFO:  common: 33:  YAML file 'params.yaml' loaded successfully]
[ 2023-12-30 12:35:19,867:  INFO:  common: 53:  Created directory at : artifacts]
[ 2023-12-30 12:35:19,871:  INFO:  common: 53:  Created directory at : artifacts\model_evaluation]


[ 2023-12-30 12:35:20,519:  INFO:  common: 66:  JSON file saved at : artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2023/12/30 12:35:35 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 4
Created version '4' of model 'ElasticnetModel'.
